In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook

output_notebook()

# Espectro y Transformada de Fourier

## Espectro


- Isaac Newton llamó **espectros** a los componentes que forman la luz blanca y que normalmente no se pueden ver
- Newton mostró usando prismas que la luz blanca puede descomponerse en colores y viceverza
- Hoy entendemos que la luz como onda tiene una **frecuencia asociada** y que cada color es una frecuencia particular


<table><tr><td>

<img src="../images/fourier-newton.jpg" width="400" align="center">  

</td><td>

<img src="../images/fourier-prism.jpg" width="400" align="center">  

</td></tr></table>


Paradójicamente, Newton nunca acepto que esto se debía a la frecuencia de la radiación ya que creía en la teoría corpuscular de la luz


### Onda

- Es una perturbación que transporta energía a través del espacio
- Se describe (tipicamente) por su frecuencia (período), amplitud y fase/desfase



### Señal sinusoidal

La siguiente es una función del tiempo completamente descrita por su amplitud $A$, frecuencia $f$ y fase $\phi$

$$
s(t) = A \cos (2 \pi f t + \phi)
$$



El recíproco de la frecuencia también se llama periódo: $P= \frac{1}{f}$

Observe como cambia la señal al modificar sus parámetros

In [ ]:
t = np.linspace(0, 5, num=1000)
s = np.cos(2.0*np.pi*t)

A = Slider(start=0.1, end=2, value=1, step=.01, title="Amplitud")
f = Slider(start=0.1, end=2, value=1, step=.01, title="Frecuencia")
p = Slider(start=0, end=6.4, value=0, step=.1, title="Desfase")

source = ColumnDataSource(data=dict(t=t, s=s))
plot = Figure(y_range=(-1.5, 1.5), plot_width=550, plot_height=300)
plot.line('t', 's', source=source, line_width=3, line_alpha=0.6)
plot.xaxis[0].axis_label = 'Tiempo [s]'

callback = CustomJS(args=dict(source=source, A=A, f=f, p=p), code="""
    var t = source.data['t'];
    var s = source.data['s'];
    for (var i = 0; i < s.length; i++) {
        s[i] = A.value*Math.cos(2*Math.PI*t[i]*f.value + p.value);
    }
    source.change.emit();
""")

for widget in [A, f, p]:
    widget.js_on_change('value', callback)

show(column(A, f, p, plot))

## Componentes frecuenciales y descomposiciones armónicas

Una sinusoide que es períodica en una fracción

$$
\frac{P}{k} ~ \forall k \in \mathbb{N}
$$ 

también es periódica en $P$

En general
- Llamamos a $f_0 = 1/P$ la **frecuencia fundamental**
- Llamamos a $f_k = k/P = kf_0$ el **k-esimo armónico de $f_0$**

Si sumamos armónicos con distintas amplitudes el resultado es una nueva función periódica que tiene la misma frecuencia fundamental

En el siguiente ejemplo modifique las amplitudes de cada armónico y observe la señal resultante

$$
s(t) = A_1 \cos(2\pi f_0) + A_2 \cos(2\pi 2f_0) + A_3 \cos(2\pi 3 f_0)
$$

Note que la suma no es necesariamente sinusoidal

In [ ]:
t = np.linspace(0, 5, num=1000)
s = np.cos(2.0*np.pi*t)

f = Slider(start=0.1, end=2, value=1, step=.01, title="Frecuencia fundamental")
A1 = Slider(start=0.1, end=2, value=1, step=.01, title="Amplitud fundamental")
A2 = Slider(start=0.1, end=2, value=0, step=.01, title="Amplitud 1er armónico")
A3 = Slider(start=0.1, end=2, value=0, step=.01, title="Amplitud 2do armónico")

source = ColumnDataSource(data=dict(t=t, s=s))
plot = Figure(plot_width=550, plot_height=300)
plot.line('t', 's', source=source, line_width=3, line_alpha=0.6)
plot.xaxis[0].axis_label = 'Tiempo [s]'

callback = CustomJS(args=dict(source=source, f=f, A1=A1, A2=A2, A3=A3), code="""
    var t = source.data['t'];
    var s = source.data['s'];
    for (var i = 0; i < s.length; i++) {
        s[i] = A1.value*Math.cos(2*Math.PI*t[i]*f.value) 
        s[i] += A2.value*Math.cos(2*Math.PI*t[i]*2*f.value) 
        s[i] += A3.value*Math.cos(2*Math.PI*t[i]*3*f.value);
    }
    source.change.emit();
""")

for widget in [A1, A2, A3, f]:
    widget.js_on_change('value', callback)

show(column(f, A1, A2, A3, plot))

Podemos generalizar esta noción utilizando la **serie trigonométrica**

$$
\begin{align}
s(t) &= \sum_{k=0}^\infty A_k \cos(2\pi k f t + \phi_k) \nonumber \\
&= \sum_{k=0}^\infty a_k \cos(2\pi k f t) + b_k \sin(2\pi k f t), \nonumber
\end{align}
$$

donde $a_k = A_k \cos(\phi_k)$ y $b_k = -A_k \sin(\phi_k)$ se obtienen de $\cos(x+y) = \cos(x)\cos(y) - \sin(x)\sin(y)$

Podemos generar una función periódica arbitrarias definiendo $\{a_k, b_k\}$ y $f$

Sea por ejemplo $a_k = \forall k$ y $b_k = 1/k \forall k$

¿Qué señal se obtiene al agregar cada vez más armónicos?

In [ ]:
t = np.linspace(0, 5, num=1000)
s = np.sin(2.0*np.pi*t)

f = Slider(start=0.1, end=2, value=1, step=.01, title="Frecuencia fundamental")
K = Slider(start=1, end=50, value=1, step=1, title="Armónicos")

source = ColumnDataSource(data=dict(t=t, s=s))
plot = Figure(plot_width=550, plot_height=300)
plot.line('t', 's', source=source, line_width=3, line_alpha=0.6)
plot.xaxis[0].axis_label = 'Tiempo [s]'

callback = CustomJS(args=dict(source=source, f=f, K=K), code="""
    var t = source.data['t'];
    var s = source.data['s'];
    for (var i = 0; i < s.length; i++) {
        s[i] = 0
        for (var k = 1; k < K.value+1; k++) {
            s[i] += Math.sin(2*Math.PI*t[i]*k*f.value)/k
        }
    }
    source.change.emit();
""")

for widget in [f, K]:
    widget.js_on_change('value', callback)

show(column(f, K, plot))

<div class="alert alert-info">
    
**Notemos que:** Hasta ahora hemos visto como sintetizar señales a partir de sus armónicos, pero también podemos hacer el proceso inverso, es decir encontrar los armónicos de una señal dada
    
</div>

Tal como Newton descompuso la luz blanca en colores, nosotros podemos descomponer una señal en sus armónicos usando la **serie de Fourier**

### Apéndice: Números complejos

Sea z un número complejo, lo podemos escribir en forma cartesiana 

$$
z = \Re[z] + j \Im[z] = a + j b
$$

donde $a \in \mathbb{R}$, $b \in \mathbb{R}$ y $j = \sqrt{-1}$ es el número imaginario.


También podemos escribirlo en forma polar

$$
z = c e^{j\phi} = c \cos(\phi) + j c \sin(\phi)
$$

donde 
- $c = |z| = \sqrt{a^2 + b^2} \in [0, \infty]$ es la magnitud 
- $\phi = \angle z = \tan^{-1} \left (\frac{b}{a} \right) \in [-\frac{\pi}{2}, \frac{\pi}{2}]$ es el ángulo
- $a = c \cos(\phi)$
- $b = c\sin(\phi)$

se pueden escribir las siguientes relaciones 

$$
\cos(\phi) = \frac{1}{2} (e^{j\phi} + e^{-j\phi}) ~\wedge~ \sin(\phi) = \frac{1}{2j} (e^{j\phi} - e^{-j\phi})
$$

el complejo conjugado de $z = a + j b = c e^{j\phi}$ es

$$
z^* = a - jb = c e^{-j\phi}
$$

## Serie de Fourier


- En 1807 *Jean Baptiste Joseph Fourier* presenta un teorema indicando que una función periódica arbitraria con periódo $P=1/f_0$ puede representarse como una suma ponderada de senos y cosenos
- La serie de Fourier (FS) es una generalización de la serie trigonométrica a los números complejos

$$
s(t) = \sum_{k=-\infty}^{\infty} c_k e^{j 2\pi k f_0 t}, ~~ c_k \in \mathbb{C}
$$



### Apéndice: Bases generadoras

- Una base es un conjunto de un espacio vectorial
 - Los elementos de la base son linealmente independientes
 - Todos los elementos del espacio pueden expresarse como una combinación lineal de la base
 
Considere el espacio vectorial $\mathbb{R}^3$. El siguiente conjunto 

$$
\left \{ v_1=\begin{pmatrix} 1 \\ 0 \\ 0 \end{pmatrix}, v_2= \begin{pmatrix} 0 \\ 1 \\ 0 \end{pmatrix}, v_3=\begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix}\right \},
$$

es una base generadora de $\mathbb{R}^3$, es decir un vector cualquiera $\vec a = (a_1, a_2, a_3) \in \mathbb{R}^3$ se puede representar como 

$$
\vec a = a_1 v_1 + a_2 v_2 + a_3 v_3.
$$

Un conjunto ortonormal siempre es linealmente independiente


### Base de Fourier

El conjunto de funciones 

$$
v_k (t) = \frac{1}{\sqrt{P}} e^{j2\pi k t / P} ~~ \forall k \in \mathbb{Z}
$$

cumple 

$$
\langle v_n (t), v_m (t) \rangle = \int_0^P v_n (t) v_m^* (t) dt = \frac{1}{P} \int_0^P e^{j2\pi (n-m)t/P} dt =\begin{cases}1 & n=m \\ 0 & n \neq m\end{cases}
$$

*i.e.* es un conjunto ortonormal (l.i.) en el espacio de funciones periódicas con periódo $P$

Esto es facilmente comprobable si estudiamos

$$
\int_0^P e^{j2\pi k t / P} dt = \int_0^P \cos(2\pi k t/P) dt + j \int_0^P \sin(2\pi k t/P) dt 
$$

Es decir que

$$
\begin{align}
\langle s(t),  e^{j 2\pi m t/ P} \rangle &= \int_0^P s(t) e^{-j 2\pi m t/ P} dt \nonumber \\
&= \int_0^P \sum_{k=-\infty}^{\infty} c_k e^{j 2\pi k t/P} e^{-j 2\pi m t/P} dt \nonumber \\
&= \sum_{k=-\infty}^{\infty} c_k  \int_0^P e^{j 2\pi (k-m) t/P}  dt \nonumber \\
&=  c_m  P \nonumber \\
\end{align}
$$

Por lo que podemos encontrar los coeficientes de la FS  usando

$$
c_m = \frac{1}{P} \int_0^P s(t) e^{-j 2\pi m t/P} dt
$$


### Ejemplo: FS de señal cuadrada

Sea 

$$
s(t) = \begin{cases} 1 & t \in[0, \frac{P}{2}] \\ 0 & t \in [\frac{P}{2}, P] \end{cases}
$$

Los coeficientes de su FS son

$$
c_0 = \frac{1}{P} \int_0^P s(t) dt = \frac{1}{P} \int_0^{P/2} dt = \frac{1}{2}
$$

y

$$
\begin{align}
c_k &= \frac{1}{P} \int_0^\frac{P}{2} e^{-j2\pi kt/P} dt  \nonumber  \\  
&= -\frac{j}{P}  \int_0^\frac{P}{2} \sin(2\pi kt/P) dt \nonumber \\
&=  0 + j \frac{\cos(\pi k)  - 1}{2\pi k} \nonumber 
\end{align}
$$

Notemos que los coeficientes 
- A excepsión de $c_0$ sólo tienen parte imaginaria
- sólo son distintos de cero para armónicos impares (seno)

Finalmente la FS está dada por

$$
\begin{align}
s(t) &= \sum_{k=-\infty}^{\infty} j \frac{\cos(\pi k)  - 1}{2\pi k}  e^{j 2\pi k t/P} \nonumber \\
&= \frac{1}{2} + \sum_{k=1}^{\infty}  \frac{1 - \cos(\pi k)}{\pi k} \sin(2\pi k t/P)  \nonumber 
\end{align}
$$

In [ ]:
t = np.linspace(0, 5, num=1000)
s = np.sin(2.0*np.pi*t)

f = Slider(start=0.1, end=2, value=1, step=.01, title="Frecuencia fundamental")
K = Slider(start=1, end=50, value=1, step=1, title="Armónicos")

source = ColumnDataSource(data=dict(t=t, s=s))
plot = Figure(plot_width=550, plot_height=300)
plot.line('t', 's', source=source, line_width=3, line_alpha=0.6)
plot.xaxis[0].axis_label = 'Tiempo [s]'

callback = CustomJS(args=dict(source=source, f=f, K=K), code="""
    var t = source.data['t'];
    var s = source.data['s'];
    for (var i = 0; i < s.length; i++) {
        s[i] = 0.5
        for (var k = 1; k < K.value+1; k++) {
            s[i] += (1-Math.cos(Math.PI*k))*Math.sin(2*Math.PI*t[i]*k*f.value)/(Math.PI*k)
        }
    }
    source.change.emit();
""")

for widget in [f, K]:
    widget.js_on_change('value', callback)

show(column(f, K, plot))

### Propiedades de la FS

- Si $s(t)$ es par entonces $c_k$ es par
- Si $s(t)$ es impar entonces $c_k$ es impar
- Si $s(t + P/2) = -s(t)$ (antiperiódica) entonces $c_k=0$ para k par
- Si $s(t)$ es real y par entonces $c_k$ es real y par
- Si $s(t)$ es real e impar entonces $c_k$ es imaginario e impar
- La FS es lineal


### Teorema de Parseval

La potencia de una señal puede calcularse a partir de sus componentes

$$
\begin{align}
P_s &= \frac{1}{P} \int_0^P |s(t)|^2 dt \nonumber \\
&= \frac{1}{P} \int_0^P |\sum_{k=-\infty}^{\infty} c_k e^{j 2\pi k t/P }|^2 dt \nonumber \\
&= \frac{1}{P} \int_0^P \sum_{k=-\infty}^{\infty} |c_k |^2 dt \nonumber \\
&= \sum_{k=-\infty}^\infty |c_k|^2 \nonumber
\end{align}
$$

## Transformada de Fourier

<div class="alert alert-info">
    
**Notemos que:** Podemos extender el concepto de descomposición armónica a señales no periódicas usando la Transformada de Fourier. Esta herramienta matemática será fundamental en este curso
    
</div>

- El concepto de frecuencia puede aplicarse también a señales no-periódicas
- Según **Joseph Fourier** una señal no-periódica puede ser vista como una señal periódica **con un período infinito**
- El único requisito es que ahora las frecuencias son un continuo, con un espaciado infinitesimal


Veremos que:
- Una señal analógica puede ser vista como continua en el tiempo o continua en frecuencia
- Más adelante estudiaremos la Transformada de Fourier discreta (DFT) para señales digitales


### Derivación a partir de la FS

Sea un tren de pulsos cuadrado con periódo P y ancho $2T < P$ definido en un período como

$$
s(t) = \begin{cases} 1, & |t| < T \\ 0, & T<|t| < P/2 \end{cases}
$$

Su serie de Fourier es 

$$
c_0 = \frac{1}{P} \int_{-P/2}^{P/2} s(t) dt = \frac{1}{P} \int_{-T}^{T}  dt = \frac{2T}{P}
$$

y

$$
\begin{align}
c_k &= \frac{1}{P} \int_{-T}^{T} e^{-j2\pi kt/P} dt  \nonumber \\
&= \frac{1}{\pi k} \sin \left (2\pi k \frac{T}{P} \right)  \nonumber 
\end{align}
$$


En el siguiente ejemplo la figura izquierda muestra la señal de pulso cuadrado mientras que la derecha muestra $c_k$ en función de $k$

¿Qué ocurre con $c_k$ a medida que $P$ aumenta ($f$ disminuye)?

In [ ]:
t = np.linspace(-3, 3, num=1000)
s = np.zeros_like(t)
k = np.arange(-100, 100)
c = np.zeros_like(k)

f = Slider(start=0.1, end=1, value=1, step=.01, title="Frecuencia fundamental")

source_s = ColumnDataSource(data=dict(t=t, s=s))
source_k = ColumnDataSource(data=dict(k=k, c=c))
plot1 = Figure(plot_width=300, plot_height=300)
plot1.line('t', 's', source=source_s, line_width=3, line_alpha=0.6)
plot2 = Figure(plot_width=300, plot_height=300)
plot2.scatter('k', 'c', source=source_k, line_width=3, line_alpha=0.6, legend_label='P x ck')
plot2.line('k', 'c', source=source_k, line_width=3, line_alpha=0.6, legend_label='envolvente')
plot1.xaxis[0].axis_label = 'Tiempo [s]'
plot2.xaxis[0].axis_label = 'k'

callback = CustomJS(args=dict(source_s=source_s, source_k=source_k, f=f), code="""
    var t = source_s.data['t'];
    var s = source_s.data['s'];
    var c = source_k.data['c']
    var T = 0.25;
    c[100] = 2*T;
    for (var k = 1; k < 100; k++){
        c[100+k] = (Math.sin(2*Math.PI*k*T*f.value))/(Math.PI*k*f.value);
        c[100-k] = (Math.sin(2*Math.PI*k*T*f.value))/(Math.PI*k*f.value);
    }
    for (var i = 0; i < s.length; i++) {
        s[i] = c[100];
        for (var k = 1; k < 100; k++) {
            s[i] += 2*c[100+k]*Math.cos(2*Math.PI*t[i]*k*f.value);
        }
    }    
    source_s.change.emit();
    source_k.change.emit();
""")

for widget in [f]:
    widget.js_on_change('value', callback)

layout = column(f, row(plot1, plot2))
show(layout)


- A mayor P disminuye la frecuencia fundamental
- Cuando P es muy grande el tren de pulsos tiende a un único pulso
- Llamamos "envolvente" a la linea imaginaria que conecta los coeficientes $P \cdot c_k$
- A mayor P la envolvente se muestrea con detalles más finos


Si tomamos el caso $P \to \infty$ la envolvente resultante es

$$
\lim_{P\to \infty} P c_k = S(f) = \int_{-\infty}^{\infty} s(t) e^{-j 2\pi t  f} dt,
$$

o también

$$
S(\omega) = \int_{-\infty}^{\infty} s(t) e^{-j\omega t } dt,
$$

donde $\omega = 2\pi f$ se llama frecuencia angular.

<div class="alert alert-info">
    
**Atención**: Esto se conoce como **transformada de Fourier directa** o **integral de Fourier**.

</div>

Reemplazando el resultado anterior en la ecuación de síntesis y tomando el límite cuando $f_0 \to 0$ 

$$
\begin{align}
s(t) &= \lim_{f_0 \to 0} \sum_{k=-\infty}^{\infty} f_0 S(k f_0) e^{j 2\pi t  k f_0} \nonumber  \\
&= \int_{-\infty}^{\infty} S(f) e^{j 2\pi t  f} df \nonumber \\
&= \frac{1}{2\pi} \int_{-\infty}^{\infty} S(\omega) e^{j \omega t } d\omega, \nonumber 
\end{align}
$$

<div class="alert alert-info">
    
**Atención**: Esto se conoce como **transformada de Fourier inversa**

</div>


### Par de Fourier

Los operadores  

$$
S(\omega) = \mathbb{FT} [s(t)] = \int_{-\infty}^{\infty} s(t) e^{-j\omega t } dt
$$

y

$$
s(t) = \mathbb{FT}^{-1} [S(\omega)] = \frac{1}{2\pi} \int_{-\infty}^{\infty} S(\omega) e^{j \omega t } d\omega, \nonumber 
$$

se conoce como par de Fourier y nos permiten analizar una señal en el dominio del tiempo o en el dominio de la frecuencia sin pérdidas

### Ejemplo

Consideremos nuevamente el pulso cuadrado

$$
s(t) = \begin{cases} 1, & |t| < T \\ 0, & |t| > T\end{cases}
$$

su transformada de Fourier es

$$
\begin{align}
S(\omega) &= \int_{-\infty}^{\infty} s(t) e^{-j\omega t } dt \nonumber \\
&= \int_{-T}^{T} e^{-j\omega t } dt \nonumber \\
&= \frac{1}{-j\omega} \left(e^{-j\omega T }  - e^{j\omega T } \right) \nonumber \\
&= \frac{2}{\omega}  \sin(\omega T) = 2T \text{sinc}(\omega T) \nonumber 
\end{align}
$$

Esto calza precisamente con lo que vimos anteriomente

In [ ]:
T = 0.25

w = np.arange(-100, 100)
S = 2*np.sin(w*T)/w
S[w==0] = 0.5

source_S = ColumnDataSource(data=dict(w=w, S=S))
p = Figure(plot_width=400, plot_height=300)
p.line('w', 'S', source=source_S, line_width=3, line_alpha=0.6)
p.xaxis[0].axis_label = 'Frecuencia angular'
p.yaxis[0].axis_label = 'Espectro'

show(p)

### Propiedades de la transformada de Fourier


1) La transformada de Fourier es un operador lineal, si tenemos dos señales y dos valores escalares entonces

$$
\mathbb{FT}[c_1 s_1(t) + c_2 s_2(t)] = c_1\mathbb{FT}[s_1(t)] + c_2\mathbb{FT}[s_2(t)] 
$$

2) El operador de convolución en el tiempo se convierte en multiplicación en frecuencia 

$$
\mathbb{FT}[(s_1 * s_2)(t)] =  \mathbb{FT}[s_1(t)] \cdot \mathbb{FT}[s_2(t)],
$$

donde 

$$
(s_1 * s_2)(t) = \int s_1(\tau) s_2(t-\tau) d\tau
$$

es la operación de convolución

3) Así mismo, la multiplicación en frecuencia se convierte en multiplicación en el tiempo

$$
\mathbb{FT}[s_1(t)\cdot s_2(t)] =  \frac{1}{2\pi}\mathbb{FT}[s_1(t)] * \mathbb{FT}[s_2(t)]  
$$

### Teorema de Parseval

La energía se preserva entre frecuencia y tiempo
$$
\int | s(t) |^2 dt = \frac{1}{2\pi} \int | S(\omega) |^2 d\omega
$$


### Definición: Espectros de amplitud y fase

Sea la transformada de Fourier de una función general $s(t)$

Llamamos a $S(\omega)$ el espectro de $s(t)$ 

El espectro es un número complejo que podemos escribir en notación polar como

$$
S(\omega) = |S(\omega)| e^{j\Phi(\omega)},
$$

donde $|S(\omega)|$ se conoce como **espectro de amplitud** y $\Phi(\omega)$ como **espectro de fase**

## Transformada de Fourier Discreta (DFT)

En nuestro caso particular nos interesa procesar señales usando software

Por ende asumiremos que existe un proceso que sensa y convierte la señal analógica continua en una señal digital, como en el siguiente diagrama

<img src="../images/signal-sampling1.png" width="500">

- La señal análogica $s(t)$ se muestrea cada $T_s$ durante un tiempo $T$
- El tiempo asociada a $s_n$ es $t_n = n \cdot T_s$ [s]
- El inverso del periódo de muestreo se denomina **frecuencia de muestreo**: $F_s = \frac{1}{T_s}$ [Hz]
- La cantidad de muestras del arreglo $\{s_n\}$ es equivalente a $N = \lfloor T F_s \rfloor $

Más formalmente diremos que existe un sistema muestreador con frecuencia de muestreo $F_s$ [Hz] tal que

$$
s(t) = \sum_{n=0}^{N-1} s[n] \delta(t - n/F_s),
$$

Si reemplazamos en la transformada de Fourier obtenemos

$$
\begin{align}
S(\omega) &= \int s(t) e^{-j\omega t} dt \nonumber \\
&= \int \sum_{n=0}^{N-1} s[n] \delta(t - n/F_s) e^{-j\omega t} dt \nonumber \\
&=  \sum_{n=0}^{N-1} s[n] \int \delta(t - n/F_s) e^{-j\omega t} dt \nonumber \\
&=  \sum_{n=0}^{N-1} s[n] e^{-j\omega n/F_s} \nonumber 
\end{align}
$$

Finalmente definiendo $\omega = 2 \pi f = 2 \pi k \Delta f$ donde $\Delta f = \frac{1}{T} = \frac{F_s}{N}$ y reemplazando obtenemos

$$
S[k] =  \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n},
$$

donde $k = [0, 1, \ldots N-1]$


<div class="alert alert-info">
    
**Atención**: Esto se conoce como **transformada de Fourier discreta** o DFT

</div>

- El índice $n$ representa la discretización en el tiempo
- El índice $k$ representa la discretización en frecuencia

### Propiedades de la DFT

1) La DFT comparte las propiedades de la FT

2) La DFT es periódica, con período $N$

$$
\begin{align}
S[k+N] &= \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} (k+N) n} \nonumber \\
&=   \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n} e^{-j 2 \pi n} \nonumber \\
&= \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n} \nonumber 
\end{align}
$$

### Definición: Frecuencia de Nyquist

Es la frecuencia más alta con que puede representarse una señal muestreada a $F_s$

Su valor es $\frac{F_s}{2}$


### Rango frecuencial de la DFT

Considerando la frecuencia de Nyquist y la periodicidad de la DFT la correspondencia entre índices y frecuencias es

$$
\begin{matrix}
k &  : & 0, & 1, & 2, & \ldots & N/2 -1, & N/2, & N/2+1, & \ldots & N-2, & N-1 \\
f = k \frac{F_s}{N} & : & 0, & \frac{F_s}{N}, & \frac{2 F_s}{N}, & \ldots & \frac{F_s}{2} - \frac{F_s}{N}, & \frac{F_s}{2} = -\frac{F_s}{2}, &  \frac{F_s}{2} + \frac{F_s}{N} = - \frac{F_s}{2}  + \frac{F_s}{N}, & \ldots & -\frac{2 F_s}{N},  & -\frac{F_s}{N}
\end{matrix}
$$

### La DFT es  un producto matricial

Sea $\{s_n\}_{n=0,\ldots,N-1}$ y definiendo 

$$
W_N = e^{-j \frac{2\pi}{N}} = \cos \left(\frac{2\pi}{N}\right) - j \sin \left(\frac{2\pi}{N}\right)
$$

podemos expresar la transformada de Fourier discreta como

$$
S[k] =  \sum_{n=0}^{N-1} s[n] W_N^{kn}, \quad k = [0, 1, \ldots N-1],
$$

que también puede ser expresado matricialmente como

$$
\begin{align}
\begin{pmatrix} 
S[0] \\
S[1] \\
S[2] \\
\vdots \\
S[N-1] \\
\end{pmatrix} &=
\begin{pmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & W_N & W_N^2 & \cdots & W_N^{N-1} \\
1 & W_N^2 & W_N^4 & \cdots & W_N^{N-2} \\
\vdots & \dots & \dots & \ddots &  \vdots \\
1 & W_N^{N-1} & W_N^{N-2} & \cdots & W_N \\
\end{pmatrix} 
\begin{pmatrix} 
s[0] \\
s[1] \\
s[2] \\
\vdots \\
s[N-1] \\
\end{pmatrix} \nonumber  \\
S &= \Omega s,
\end{align}
$$


Notemos que:
- Por definición $W_N^{kn} = \left(e^{-j \frac{2\pi}{N}}\right)^{kn} = e^{-j \frac{2\pi}{N}kn}$
- Por periodicidad $W_N^{2(N-1)} = W_N^{2(N-1) - N}  = W_N^{N-2}$
- También se tiene simetría hermítica: $W_N^{k(-n)} = W_N^{-kn} = (W_N^{kn})^*$
- $\Omega$ es una matriz cuadrada y simétrica 

<div class="alert alert-info">
    
**Costo computacional:** El cálculo de la DFT tiene complejidad cuadrática: $N^2$ multiplicaciones y $N$ sumas

</div>




### Ejemplo

Sea $S=\{S[0], S[1], S[2], S[3]\}^T$ podemos encontrar $s$ usando

$$
S= 
\begin{pmatrix} 
S[0] \\
S[1] \\
S[2] \\
S[3] 
\end{pmatrix} =
\begin{pmatrix}
1 & 1  & 1 & 1\\
1 & W_4  & W_4^2 & W_4^3 \\
1 & W_4^2  & W_4^4 & W_4^2 \\
1 & W_4^3  & W_4^2 & W_4 \\
\end{pmatrix} 
s = \begin{pmatrix}
1 & 1  & 1 & 1\\
1 & -j  & -1 & j \\
1 & -1  & 1 & -1 \\
1 & j  & -1 & -j \\
\end{pmatrix} 
s
$$

Luego sólo tenemos que encontrar el inverso de $\Omega$

In [ ]:
np.set_printoptions(precision=4, suppress=True)

W4 = np.exp(-1j*2*np.pi/4)
index = np.array([0, 1, 2, 3])[:, np.newaxis]
Omega = W4**(index*index.T)

print(Omega)

np.linalg.inv(Omega) # Estación función calcula el inverso de una matriz cuadrada

Notemos que el inverso de $\Omega$ es su complejo conjugado dividido N

$$
\Omega^{-1} = \frac{1}{4}
\begin{pmatrix}
1 & 1 & 1 &  1 \\
1 & j & -1 &  -j \\
1 & -1 & 1 &  -1 \\
1 & -j & -1  & j \\
\end{pmatrix} = \frac{1}{4} \Omega^*
$$

Se puede verificar que esto ocurre para todo N

### DFT inversa

En resumen podemos recuperar $s$ a partir de $S$ usando

$$
s = \frac{1}{N} \Omega^* S
$$

o lo que es equivalente

$$
s[n] = \frac{1}{N} \sum_{k=0}^{N-1} S[k] W_N^{-kn}, \quad n = [0, 1, \ldots N-1]
$$

que corresponde a la DFT inversa y cuya únicas diferencias con la DFT son el factor $\frac{1}{N}$ y el signo del exponente

## Transformada Rápida de Fourier (FFT)

- La computación de la DFT tiene complejidad $\mathcal{O}(N^2)$, es muy costosa de aplicar en la práctica
- Existe una aproximación numérica exacta con complejidad $\mathcal{O}(N\log N)$: la **Fast Fourier Transform** (FFT). 

El algoritmo de Cooley-Tukey, que es tal vez el algoritmo FFT más famoso, obtiene una expresión recursiva que explota las simetrías en la DFT

$$
\begin{align}
S[k] &=  \sum_{n=0}^{N-1} s[n] W_N^{kn} \nonumber \\
&= \sum_{n=0}^{N/2-1} s[2n] W_N^{k 2n} + \sum_{n=0}^{N/2-1} s[2n+1] W_N^{k(2n+1)} \nonumber \\
&= \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{kn} + W_N^{k} \sum_{n=0}^{N/2-1} s[2n+1] W_{N/2}^{kn} \nonumber \\
&= S_E[k] + W_N^{k} S_O[k] ~~ \forall k \in [0,N/2]  \nonumber 
\end{align} 
$$

Notar que se calculan dos "medias" DFT, una mitad se enfoca en los índices pares (even) y la otra en los impares (odd)

Por periodicidad de la DFT tenemos que
$$
\begin{align}
S_E[k + N/2] &=  \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{(k+N/2)n} \nonumber \\
&=  \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{kn} \exp \left(-j2\pi n \right) = S_E[k], \nonumber
\end{align}
$$

e igualmente

$$
S_O[k + N/2] = S_O[k],
$$

juntando ambos tenemos que
$$
\begin{align}
S[k + N/2] &=  S_E[k + N/2] + W_{N}^{(k+N/2)} S_O[k + N/2] \nonumber  \\
&=  S_E[k] + W_{N}^{k} \exp \left(-j\pi\right) S_O[k] \nonumber \\
&=  S_E[k] - W_{N}^{k} S_O[k] \nonumber 
\end{align}
$$

es decir

$$
\begin{align}
S[k] &=  S_E[k] + W_{N}^{k} S_O[k] \nonumber \\
S[k + N/2] &=  S_E[k] - W_{N}^{k} S_O[k] \quad \forall k \in [0,N/2]  \nonumber 
\end{align}
$$


<div class="alert alert-info">
    
**Costo computacional:** La DFT de $k$ y $k+N/2$ difieren sólo en un signo. Cada ves que dividimos el intervalo podemos reducir los cómputos a la mitad. El costo de la FFT tiene complejidad $N \log N$

</div>


<img src="../images/fft-16samples.png">

Imagen tomada de: http://www.themobilestudio.net/the-fourier-transform-part-14

Cantidad de multiplicaciones

| N | DFT | FFT | FFT/DFT [%] |
|---|---|---|---|
| 32 | 1024 | 160 | 15.6 |
| 128 | 16,384 | 896 | 5.46 |
| 1,024 | 1,048,576 | 10,240 | 0.97 |

## FFT en Python

Existen múltiples implementaciones de la FFT. En este curso utilizaremos el módulo `fft` de la librería científica `scipy`: https://docs.scipy.org/doc/scipy/reference/tutorial/fft.html

Si se require mayor eficiencia se puede considerar la librería [pyFFTW](https://hgomersall.github.io/pyFFTW/) que es un wrapper de la Fast Fourier Transform in the West (FFTW), famosa implementación escrita en lenguaje C

### Funciones básicas de `scipy.fft`

Las principal función que utilizaremos es

```python
scipy.fft.fft(x, # Un arreglo de NumPy,
              n=None, # El número de muestras que se usarán para calcular la FFT
              axis=-1, # El eje del arreglo en que se calculara la FFT,
              workers=None, # Número de trabajadores para calcular la FFT en paralelo
              ...)
```

que calcula la FFT de un arreglo de NumPy. Por convención, esta función retorna un arreglo con el espectro de frecuencia ordenando según 

$$
f = \left[0, \frac{F_s}{N},  \frac{2 F_s}{N}, \ldots, \frac{F_s}{2},  \ldots  -\frac{2 F_s}{N},   -\frac{F_s}{N} \right]
$$ 

es decir que primero retorna las frecuencias positivas y luego las negativas en orden invertido

Podemos usar la función


```python
scipy.fft.ftshift(x, #  Un arreglo de numpy
                  ...
                 )
```

para dar vuelta la primera y segunda mitad del espectro obteniendo un orden más natural

También podemos usar

```python
scipy.fft.fftfreq(n, #  El número de muestras de x
                  d=1.0 # El inverso de la frecuencia de muestreo
                 )
```

para calcular las frecuencias asociadas a la FFT (útil para graficar)


### Ejemplo

Sea una señal con dos componentes sinusoidales a frecuencia $1$ y $4$, respetivamente, y con desfase $0$ y $\pi/3 \approx 1.0472$, respectivamente

Calculemos los espectros de magnitud y fase usando la FFT

In [ ]:
import scipy.fft as sfft

Fs = 40
t = np.arange(0, 5, step=1./Fs); 
s = np.cos(2.0*np.pi*t) + 0.25*np.cos(2.0*np.pi*4*t + np.pi/3)
f = sfft.fftshift(sfft.fftfreq(n=len(t), d=1./Fs)); 
S = sfft.fftshift(sfft.fft(s)); 
SA = np.absolute(S)
SP = np.angle(S)

source = ColumnDataSource(data=dict(t=t, s=s, f=f, SA=SA, SP=SP))
p1 = Figure(plot_width=600, plot_height=150)
p2 = Figure(plot_width=300, plot_height=200)
p3 = Figure(plot_width=300, plot_height=200)
p1.line('t', 's', source=source, line_width=3, line_alpha=0.75)
p2.line('f', 'SA', source=source, line_width=3, line_alpha=0.75)
p3.line('f', 'SP', source=source, line_width=3, line_alpha=0.75)
p1.xaxis[0].axis_label = 'Tiempo [s]'
p2.xaxis[0].axis_label = 'Frecuencia [Hz]'
p3.xaxis[0].axis_label = 'Frecuencia [Hz]'
show(column(p1, row(p2, p3)))

Considerando sólo las frecuencias positivas (mitad derecha del espectro) podemos buscar la frecuencia de los dos puntos más altos del espectro de magnitud

In [ ]:
mask = np.argsort(SA[f>0])[-2:] # Ordena de menor a mayor y me quedo con las últimas dos
f[f>0][mask]

Las fases asociadas a estas frecuencias son

In [ ]:
SP[f>0][mask]

<div class="alert alert-info">
    
Logramos recuperar las frecuencias y las fases de los componentes individuales

</div>

Comparemos ahora la diferencia en tiempo de cómputo entra la DFT calculada como producto matricial y la FFT

In [ ]:
tf = t[:, np.newaxis]*f[:, np.newaxis].T
print("DFT numpy")
%timeit -n 5 -r 5 np.dot(s, np.exp(-1j*2.0*np.pi*tf));
print("FFT fftpack")
%timeit -n 5 -r 5 sfft.fftshift(sfft.fft(s))

## Actividades a realizar en la sesión práctica

- Profundizar en el uso de `scipy.fft`
- Calcular el espectro de ruido blanco y ruido rojo